
# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**（関数名・引数名・返却列・順序）
* I O 禁止、不要な `print` や `sort_values` 禁止

---

## 1) 問題

* `{{PROBLEM_STATEMENT}}`
  「`Product` に存在する **全ての `product_key`** を購入した `customer_id` を返せ」
* 入力 DF: `{{INPUT_DATAFRAMES}}`
  `Customer(customer_id:int, product_key:int)`（重複行あり／`customer_id` 非NULL）
  `Product(product_key:int)`（主キー）
* 出力: `{{OUTPUT_COLUMNS_AND_RULES}}`
  列: `customer_id` のみ（順序任意、重複なし）

---

## 2) 実装（指定シグネチャ厳守）

> 原則は **列最小化 → ユニーク化 → セミジョイン（存在する product のみ） → 顧客ごと集計 → 条件抽出**。

```python
import pandas as pd

def customers_bought_all_products(customer: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['customer_id']
    """
    # 全商品のユニーク数（PKだが安全のため nunique）
    total_products = product['product_key'].nunique()

    # Product が空なら（除算の公理より）全顧客が条件を満たす
    if total_products == 0:
        return customer[['customer_id']].drop_duplicates()

    # Product に存在する product_key のみに限定（セミジョイン）
    valid_keys = product['product_key']
    pairs = customer.loc[
        customer['product_key'].isin(valid_keys) & customer['product_key'].notna(),
        ['customer_id', 'product_key']
    ].drop_duplicates()

    # 顧客ごとに「購入ユニーク数」を集計し、全商品数と一致する顧客を抽出
    bought_cnt = pairs.groupby('customer_id')['product_key'].nunique()
    keep_ids = bought_cnt.index[bought_cnt.eq(total_products)]

    # 指定の列・順序で返却（順序は任意）
    out = pd.DataFrame({'customer_id': keep_ids})
    return out

Analyze Complexity
Runtime 294 ms
Beats 41.27%
Memory 67.23 MB
Beats 98.63%

```

* 追加メモ

  * 返却列は **`['customer_id']` のみ**。並びは任意なので `sort_values` 不要。
  * `drop_duplicates` により `Customer` 側の重複行の水増しを防止。
  * `isin(valid_keys)` で **Product に存在するキーのみ**を対象化（正しさ担保）。

---

## 3) アルゴリズム説明

* 使用API:

  * `Series.nunique()`：全商品数の算出
  * `Series.isin()`：セミジョイン（`Product` に存在するキーに限定）
  * `DataFrame.drop_duplicates()`：`(customer_id, product_key)` の重複排除
  * `GroupBy.nunique()`：顧客ごとのユニーク購入数
* **NULL / 重複 / 型**

  * `product_key` の NULL は対象外（`notna()` で明示）
  * `Customer` の重複は `(customer_id, product_key)` 単位で除去
  * 型は整数想定。混在がある場合は事前に `astype('Int64')` 等で正規化

---

## 4) 計算量（概算）

* `isin`：**O(N)**（ハッシュセット化）
* `drop_duplicates`：**O(N)** 〜 **O(N log N)**
* `groupby().nunique()`：**O(N)** 近似（ハッシュ集約）
* メモリ：ユニークな `(customer_id, product_key)` の一時保持分

---

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[Customer] --> B[列最小化 customer_id, product_key]
  C[Product]  --> D[全商品数 nunique]
  B --> E[Product に存在する key でフィルタ]
  E --> F[drop_duplicates で重複除去]
  F --> G[groupby customer_id で nunique]
  D --> H[全商品数と一致で抽出]
  G --> H
  H --> I[出力 customer_id]
```

---

### 実務Tips（大型データ向け）

* `valid_keys = product['product_key'].unique()` として numpy 配列にしてから `isin` してもOK（等価）。
* さらにメモリを詰める場合、`pairs` を作らず `groupby('customer_id')['product_key'].nunique()` を直接当てても良いですが、その場合も **Product に存在する key のみ**へ先に絞ることが重要です。

良い数値です（特にメモリ 👍）。まだ **実行時間は縮みます**。主因は `isin` と `nunique` のハッシュ集約コストなので、ここを削ると効きます。

要点だけ先に：

* `isin` を **カテゴリ化（辞書化）** で代替 → O(1) 判定に
* `nunique` を **`drop_duplicates` → `value_counts`** に置換（既にユニーク化済なら `nunique` は不要）
* スカラ分岐（`Product` が空なら全員合格）を先に処理

以下、**置き換え版（同シグネチャ）** です。

```python
import pandas as pd

def customers_bought_all_products(customer: pd.DataFrame, product: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: 列名と順序は ['customer_id']
    """
    # 1) 全商品のユニーク数（0なら除算の公理より全顧客が合格）
    #    ※ product_key がPK前提でも nunique() で安全取得
    total_products = product['product_key'].nunique()
    if total_products == 0:
        return customer[['customer_id']].drop_duplicates()

    # 2) Customer.product_key を Product のユニーク集合でカテゴリ化
    #    → Product に存在しない key は codes = -1 になる（高速な O(1) マッピング）
    prod_keys = pd.unique(product['product_key'])
    cat = pd.Categorical(customer['product_key'], categories=prod_keys, ordered=False)
    codes = pd.Series(cat.codes, index=customer.index)  # int32、未登録は -1

    # 3) Product に存在するキーのみ残す（-1 は除外）
    mask = codes.ge(0)
    #    列最小化しつつ、(customer_id, code) の重複を先に除去
    pairs = customer.loc[mask, ['customer_id']].assign(code=codes[mask]).drop_duplicates()

    # 4) すでに (customer_id, code) はユニークなので、
    #    「顧客ごとの購入ユニーク数」は単純に value_counts で数えるだけで良い
    cnt = pairs['customer_id'].value_counts(sort=False)

    # 5) 全商品数と一致する customer_id を抽出
    keep_ids = cnt.index[cnt.eq(total_products)]

    # 6) 仕様列のみ返却（順序は任意）
    out = pd.DataFrame({'customer_id': keep_ids})
    return out

Analyze Complexity
Runtime 271 ms
Beats 79.67%
Memory 68.04 MB
Beats 37.48%

```

### なぜ速いのか（ボトルネックの潰し方）

* **`isin` → Categorical の `.codes`**
  `isin` は右辺をセット化しても各行ハッシュ照合が必要。一方で `Categorical(categories=prod_keys)` は内部辞書により **ベクトル化 O(1)** でコード化でき、未登録は **-1** に落ちるのでフィルタも速い。
* **`nunique` → `drop_duplicates` + `value_counts`**
  `(customer_id, product_key)` を重複排除済なら、あとは **顧客IDの件数**を数えるだけ。`value_counts(sort=False)` は単純なカウンタで速い。`nunique` のような追加ハッシュは不要。
* **早期リターン**（Product空集合）
  ベンチではレアでも、理論上の最適分岐は入れておくと安定する。

### 追加の微調整（必要に応じて）

* `customer` が超巨大で `customer['product_key']` に欠損が多い場合：
  `cat = pd.Categorical(customer['product_key'].dropna(), categories=prod_keys)` としてからインデックス揃えで戻すと、若干速く・軽くなります（ただし現行の `.ge(0)` フィルタで十分なことが多い）。
* `prod_keys = product['product_key'].to_numpy()` にしてもOK（`unique()` 同等）。
* もし `customer_id` が連番で密なら、`value_counts` の代わりに `groupby('customer_id', sort=False).size()` でも同等です。

### 計算量（更新版）

* カテゴリ化：**O(N)**（内部辞書参照）
* `drop_duplicates`：**O(N)**〜**O(N log N)**（実装依存）
* `value_counts`：**O(N)**
  総じて **線形近似**。現行 294ms からさらに短縮する余地があります（分布と N 次第ですが、二桁％改善は狙えます）。

---

#### まとめ

* 良い実装ですが、**`isin` と `nunique` を外す**とまだ伸びます。
* 上記の置き換え版は **I/Oなし・sortなし・列最小化** の原則を守りつつ、理論上も正しい最短手筋です。

